# Deep et Full-connected : discrimination d'une image

In [1]:
from keras import Sequential
from keras.layers import Dense
from sklearn import model_selection
import tensorflow as tf
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

#Importation des librairies
import numpy as np
import matplotlib.pyplot as plt
import os
import scipy.io as scp
import math
import pandas as pd

## Approche basee Descripteurs (basee modele)

In [2]:

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os
os.chdir("/content/drive/My Drive/EICNAM")
!ls

 Anglais		       'Projet PYTHON'
 Discrimination_image2.ipynb   'Rapport - Projet Défi EGC.gdoc'
 Discrimination_Images1.ipynb   TOEIC
 JAVA			        WangSignatures.xlsx
'Projet .gdoc'


## Calcul des descripteurs et paramètrage du model

In [4]:
def discrimination(data):
    
    #Calcul des descripteurs
    
    #Création du vecteur resultat
    target = np.zeros([1,1000])
    target = np.array([0]*100+[1]*100+[2]*100+[3]*100+[4]*100+[5]*100+[6]*100+[7]*100+[8]*100+[9]*100)
    
    
    #association des classes à chaque daframe
    data[0] = pd.to_numeric(data[0].str.rstrip('.jpg'))
    data=data.sort_values(by = 0).assign(classe=target)
    
    X = data.iloc[:,1:-1]
    XTrain,XTest,yTrain,yTest = model_selection.train_test_split(X,target,test_size=0.2,random_state=100,stratify=target)
    y_test_simple = yTest
    yTrain = tf.one_hot(yTrain, depth=10)
    yTest = tf.one_hot(yTest, depth=10)


    # Pour un modèle avec une entrée et 10 classes (classification en catégories):
    model = Sequential()
    model.add(Dense(270, activation='tanh'))
    model.add(Dense(90, activation='relu'))
    model.add(Dense(60, activation='relu'))
    model.add(Dense(30, activation='relu'))
    model.add(Dense(10, activation='softmax'))
    
    # compile the keras model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'],experimental_run_tf_function=False)
    # fit the keras model on the dataset
    model.fit(XTrain, yTrain, epochs=150, batch_size=100, validation_split=0.1, verbose=0)
    
    pred = model.predict(XTest, batch_size = 100, verbose =0)


    predictions = list()
    for i in range(len(pred)) :
        predictions.append(np.argmax(pred[i]))
    
    #Affichage matrice de confusion pour chacune des valeurs et les scores de précision associés
    print(confusion_matrix(y_test_simple, predictions))
    print(accuracy_score(y_test_simple, predictions))
    
    return accuracy_score(y_test_simple, predictions)


## Application du systeme de discrimination basee structure Full-Connected au fichier csv des mesures

In [5]:
#Tests des jeux de données
liste_donnees = list()

data_PHOG = pd.read_excel(r'WangSignatures.xlsx',sheet_name='WangSignaturesPHOG',index_col=None, header=None)
data_JCD = pd.read_excel(r'WangSignatures.xlsx',sheet_name='WangSignaturesJCD',index_col=None, header=None)
data_CEDD = pd.read_excel(r'WangSignatures.xlsx',sheet_name='WangSignaturesCEDD',index_col=None, header=None)
data_FCTH = pd.read_excel(r'WangSignatures.xlsx',sheet_name='WangSignaturesFCTH',index_col=None, header=None)
data_FuzzyColorHistogr = pd.read_excel(r'WangSignatures.xlsx',sheet_name='WangSignaturesFuzzyColorHistogr',index_col=None, header=None)



compar = {}
compar["data_PHOG"] = discrimination(data_PHOG)
compar["data_JCD"] = discrimination(data_JCD)
compar["data_CEDD"] = discrimination(data_CEDD)
compar["data_FCTH"] = discrimination(data_FCTH)
compar["data_FuzzyColorHistogr"] = discrimination(data_FuzzyColorHistogr)

[[ 8  2  1  0  0  2  2  4  1  0]
 [ 1  8  1  2  0  3  0  2  3  0]
 [ 2  4  8  2  0  2  0  0  1  1]
 [ 0  0  1 19  0  0  0  0  0  0]
 [ 0  2  1  0 15  0  0  0  2  0]
 [ 1  2  0  0  0 10  0  4  1  2]
 [ 0  0  0  0  0  0 17  0  1  2]
 [ 2  0  2  0  0  0  0 16  0  0]
 [ 2  1  2  0  0  1  1  2  9  2]
 [ 1  1  0  0  0  0  2  2  4 10]]
0.6
[[16  0  0  0  0  2  0  0  0  2]
 [ 1 17  0  0  0  0  0  0  2  0]
 [ 0  1 17  0  0  0  0  0  2  0]
 [ 0  0  1 18  0  0  0  0  0  1]
 [ 0  0  0  0 20  0  0  0  0  0]
 [ 0  0  0  0  0 18  0  1  1  0]
 [ 0  0  0  0  0  0 20  0  0  0]
 [ 1  0  0  0  0  0  0 19  0  0]
 [ 0  3  1  0  0  1  0  0 15  0]
 [ 2  0  0  0  0  1  0  0  0 17]]
0.885
[[15  0  0  0  1  2  0  0  0  2]
 [ 1 16  0  0  0  0  0  0  3  0]
 [ 0  2 13  0  1  1  0  0  3  0]
 [ 1  0  0 18  0  0  0  0  0  1]
 [ 0  0  0  0 20  0  0  0  0  0]
 [ 2  0  1  0  0 15  0  1  1  0]
 [ 0  0  0  0  0  0 20  0  0  0]
 [ 2  0  0  0  0  2  0 16  0  0]
 [ 0  4  1  0  0  0  1  0 14  0]
 [ 3  0  0  0  0  0  0  0  0 17

### Autre technique : concaténation des mesures

In [6]:
# On regroupe les 5 résultats d'analyse en un seul dataframe
dfs = [data_PHOG,data_JCD,data_CEDD,data_FCTH,data_FuzzyColorHistogr]
#par concaténation avec la librairie pandas
result = pd.concat(dfs, join='outer', axis=1)

In [7]:
result

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125
0,0,10.0,8.0,7.0,7.0,7.0,7.0,7.0,7.0,6.0,7.0,7.0,7.0,9.0,10.0,12.0,15.0,12.0,11.0,10.0,10.0,11.0,10.0,10.0,10.0,10.0,9.0,8.0,8.0,8.0,8.0,8.0,6.0,7.0,6.0,7.0,8.0,8.0,10.0,10.0,...,87.0,108.0,123.0,110.0,90.0,77.0,92.0,106.0,108.0,95.0,65.0,73.0,80.0,83.0,82.0,64.0,70.0,71.0,67.0,61.0,69.0,76.0,79.0,75.0,67.0,69.0,78.0,83.0,79.0,70.0,64.0,72.0,78.0,79.0,71.0,57.0,62.0,67.0,69.0,69.0
1,1,15.0,13.0,12.0,11.0,10.0,10.0,10.0,8.0,9.0,10.0,10.0,10.0,9.0,9.0,8.0,8.0,6.0,6.0,8.0,9.0,9.0,10.0,12.0,12.0,11.0,13.0,12.0,12.0,13.0,13.0,11.0,11.0,15.0,11.0,4.0,1.0,2.0,1.0,2.0,...,90.0,112.0,133.0,126.0,104.0,80.0,95.0,112.0,135.0,110.0,68.0,77.0,85.0,91.0,86.0,70.0,76.0,77.0,74.0,67.0,73.0,82.0,86.0,83.0,74.0,73.0,83.0,90.0,89.0,80.0,67.0,76.0,84.0,89.0,81.0,60.0,66.0,72.0,76.0,75.0
2,10,15.0,12.0,12.0,13.0,12.0,13.0,13.0,13.0,13.0,13.0,13.0,12.0,13.0,12.0,12.0,13.0,11.0,12.0,12.0,11.0,12.0,13.0,12.0,14.0,14.0,14.0,13.0,12.0,13.0,12.0,12.0,9.0,9.0,8.0,8.0,8.0,8.0,8.0,8.0,...,94.0,117.0,142.0,153.0,133.0,88.0,108.0,138.0,195.0,164.0,78.0,93.0,114.0,147.0,154.0,73.0,81.0,85.0,85.0,79.0,78.0,89.0,97.0,98.0,91.0,80.0,93.0,106.0,113.0,106.0,77.0,91.0,108.0,131.0,130.0,71.0,83.0,99.0,130.0,246.0
3,100,12.0,10.0,6.0,4.0,4.0,3.0,3.0,3.0,3.0,4.0,4.0,4.0,5.0,7.0,11.0,14.0,15.0,10.0,7.0,5.0,5.0,4.0,4.0,4.0,5.0,4.0,4.0,5.0,6.0,9.0,9.0,7.0,6.0,6.0,5.0,5.0,5.0,5.0,4.0,...,100.0,142.0,155.0,135.0,101.0,87.0,108.0,128.0,141.0,106.0,72.0,83.0,92.0,96.0,91.0,74.0,80.0,81.0,76.0,68.0,81.0,91.0,94.0,88.0,76.0,81.0,98.0,106.0,99.0,85.0,75.0,88.0,100.0,105.0,95.0,65.0,73.0,81.0,89.0,136.0
4,101,15.0,12.0,10.0,9.0,9.0,8.0,8.0,7.0,7.0,6.0,6.0,6.0,6.0,5.0,5.0,6.0,5.0,5.0,4.0,4.0,4.0,4.0,4.0,4.0,5.0,5.0,5.0,6.0,8.0,11.0,15.0,11.0,6.0,3.0,2.0,2.0,2.0,2.0,2.0,...,146.0,158.0,140.0,131.0,103.0,97.0,133.0,173.0,255.0,147.0,77.0,98.0,125.0,162.0,144.0,61.0,66.0,66.0,63.0,58.0,78.0,87.0,88.0,83.0,73.0,94.0,119.0,125.0,116.0,95.0,88.0,122.0,180.0,170.0,133.0,74.0,94.0,121.0,150.0,168.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,995,15.0,12.0,11.0,10.0,10.0,10.0,9.0,8.0,8.0,8.0,7.0,6.0,6.0,5.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,7.0,7.0,8.0,9.0,10.0,10.0,12.0,12.0,12.0,15.0,9.0,7.0,6.0,6.0,8.0,9.0,10.0,9.0,...,16.0,20.0,27.0,31.0,26.0,15.0,20.0,29.0,49.0,36.0,14.0,18.0,25.0,35.0,40.0,12.0,13.0,14.0,15.0,14.0,13.0,16.0,18.0,19.0,18.0,14.0,18.0,22.0,25.0,25.0,14.0,18.0,25.0,37.0,42.0,14.0,18.0,24.0,42.0,255.0
996,996,15.0,13.0,14.0,13.0,13.0,13.0,14.0,13.0,13.0,14.0,14.0,13.0,13.0,12.0,12.0,14.0,11.0,11.0,12.0,14.0,14.0,13.0,12.0,11.0,11.0,11.0,11.0,12.0,12.0,13.0,7.0,6.0,6.0,4.0,4.0,4.0,4.0,4.0,5.0,...,80.0,100.0,128.0,156.0,142.0,77.0,97.0,133.0,215.0,192.0,71.0,87.0,112.0,153.0,161.0,66.0,74.0,80.0,83.0,80.0,70.0,80.0,91.0,97.0,93.0,71.0,85.0,101.0,114.0,111.0,70.0,84.0,105.0,136.0,140.0,65.0,78.0,96.0,124.0,153.0
997,997,15.0,14.0,14.0,10.0,7.0,7.0,6.0,5.0,5.0,4.0,4.0,4.0,4.0,4.0,5.0,7.0,5.0,3.0,4.0,4.0,4.0,4.0,4.0,4.0,5.0,5.0,6.0,7.0,8.0,9.0,13.0,15.0,14.0,12.0,9.0,7.0,7.0,7.0,8.0,...,49.0,62.0,78.0,88.0,83.0,49.0,63.0,85.0,123.0,124.0,46.0,58.0,79.0,123.0,148.0,37.0,41.0,45.0,46.0,45.0,41.0,48.0,54.0,57.0,56.0,44.0,53.0,64.0,74.0,73.0,45.0,56.0,74.0,103.0,114.0,43.0,55.0,75.0,133.0,255.0
998,998,15.0,12.0,11.0,9.0,8.0,7.0,7.0,7.0,6.0,6.0,6.0,6.0,7.0,8.0,10.0,11.0,9.0,7.0,7.0,7.0,7.0,7.0,8.0,8.0,8.0,8.0,9.0,10.0,11.0,12.0,14.0,10.0,8.0,7.0,5.0,4.0,5.0,4.0,5.0,...,51.0,58.0,64.0,67.0,57.0,46.0,51.0,59.0,76.0,68.0,40.0,44.0,49.0,56.0,55.0,44.0

In [12]:
def discrimination(data):
    
    #Calcul des descripteurs
    
    #Création du vecteur resultat
    target = np.zeros([1,1000])
    target = np.array([0]*100+[1]*100+[2]*100+[3]*100+[4]*100+[5]*100+[6]*100+[7]*100+[8]*100+[9]*100)
    
    
    X = data.iloc[:,1:-1]
    XTrain,XTest,yTrain,yTest = model_selection.train_test_split(X,target,test_size=0.2,random_state=100,stratify=target)
    y_test_simple = yTest
    yTrain = tf.one_hot(yTrain, depth=10)
    yTest = tf.one_hot(yTest, depth=10)


    # Pour un modèle avec une entrée et 10 classes (classification en catégories):
    model = Sequential()
    model.add(Dense(270, activation='tanh'))
    model.add(Dense(90, activation='relu'))
    model.add(Dense(60, activation='relu'))
    model.add(Dense(30, activation='relu'))
    model.add(Dense(10, activation='softmax'))
    
    # compile the keras model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'],experimental_run_tf_function=False)
    # fit the keras model on the dataset
    model.fit(XTrain, yTrain, epochs=150, batch_size=100, validation_split=0.1, verbose=0)
    
    pred = model.predict(XTest, batch_size = 100, verbose =0)


    predictions = list()
    for i in range(len(pred)) :
        predictions.append(np.argmax(pred[i]))
    
    #Affichage matrice de confusion pour chacune des valeurs et les scores de précision associés
    print(confusion_matrix(y_test_simple, predictions))
    print(accuracy_score(y_test_simple, predictions))
    
    return accuracy_score(y_test_simple, predictions)


In [13]:
discrimination(result)

[[ 6 14  0  0  0  0  0  0  0  0]
 [ 1 19  0  0  0  0  0  0  0  0]
 [ 0 20  0  0  0  0  0  0  0  0]
 [ 0  9  0 10  1  0  0  0  0  0]
 [ 0  1  0  3 15  1  0  0  0  0]
 [ 0  1  0  0  7  0  6  1  5  0]
 [ 0  2  0  0  0  0 17  1  0  0]
 [ 0  3  0  0  0  0  6  6  5  0]
 [ 0  3  0  0  0  0  0  0 16  1]
 [ 0  4  0  0  0  0  0  0 15  1]]
0.45


0.45